In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm;
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sklearn

In [11]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score
from xgboost import XGBRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV

In [3]:
train =  pd.read_csv('data/preprocess/train_0910.csv')
test =  pd.read_csv('data/preprocess/test_0910.csv')

In [4]:
X = train.drop(columns = ['SalePrice'])
y = train['SalePrice']

In [5]:
from bayes_opt import BayesianOptimization

In [13]:
def xgb_bo(gamma,max_depth, subsample):
    xgb_params = {
              'gamma' : int(round(gamma)),
              'max_depth' : int(round(max_depth)),
               'subsample' : int(round(subsample)),      
              }
    xgb = XGBRegressor(**xgb_params)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size = 0.2, )
    
    xgb.fit(X_train,y_train)
    score = mean_squared_log_error(y_valid, xgb.predict(X_valid))
    return score

In [14]:
#Validation function
n_folds = 10

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [15]:
# RMSLE 정의
def rmsle(origin, pred):
    return np.sqrt(mean_squared_log_error(origin, pred))

In [16]:
xgb_parameter_bounds = {
                      'gamma' : (0,10),
                      'max_depth' : (1,3), # 나무의 깊이
                      'subsample' : (0.5,1)
                      }

In [17]:
BO_xgb = BayesianOptimization(f = xgb_bo, pbounds = xgb_parameter_bounds,random_state = 0)

In [18]:
BO_xgb.maximize(init_points = 5, n_iter = 5)

|   iter    |  target   |   gamma   | max_depth | subsample |
-------------------------------------------------------------
|  1        |  0.000270 |  5.488    |  2.43     |  0.8014   |
|  2        |  0.000246 |  5.449    |  1.847    |  0.8229   |
|  3        |  0.000180 |  4.376    |  2.784    |  0.9818   |
|  4        |  0.000259 |  3.834    |  2.583    |  0.7644   |
|  5        |  0.000272 |  5.68     |  2.851    |  0.5355   |
|  6        |  0.000280 |  5.592    |  2.749    |  0.8524   |
|  7        |  0.000179 |  2.303    |  2.915    |  0.6916   |
|  8        |  0.000201 |  5.411    |  2.106    |  0.8663   |
|  9        |  0.000173 |  1.953    |  1.551    |  0.5432   |
|  10       |  0.000225 |  3.72     |  1.231    |  0.9286   |


In [23]:
tuned_xgb_by_BO_1 =XGBRegressor(gamma = 5.68,max_depth = 3, subsample =  0.5355)
tuned_xgb_by_BO_1.fit(X,y)


#예측
pred = tuned_xgb_by_BO_1.predict(test)
pred = np.expm1(pred)

#정답파일 내보내기
sub = pd.read_csv('data/sample_submission.csv')
sub['SalePrice'] = pred
sub.to_csv('tuned_xgb_by_BO_1.csv',index = False)